In [1]:
import tensorflow as tf
import re
import os
import collections
import numpy as np
import codecs
import random
from operator import itemgetter

In [2]:
#GPU设置
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.95 #占用95%显存
session = tf.Session(config=config)
os.environ['CUDA_VISIBLE_DEVICES']="0"

## 构建词表

In [14]:
def get_vocab(input_data,min_word_freq):
    counter = collections.Counter()
    with codecs.open(input_data,"r","utf-8") as f:
        for line in f:
            line = ' '.join(re.split(' |\t|\v|\n',line))        #将数据中的空格符统一，便于后期处理(原始数据中空格符包含\t、\v等)   
            line = re.split('([: ,.(){}\[\]=])',line)        #将字符串数据按照括号中的符号进行分割，分割成列表格式，并且在列表中保留分隔符
            line = list(filter(lambda x: x!=' 'and x!='',line))
            for word in line:
                counter[word] += 1
                
        counter = filter(lambda x: x[1] > min_word_freq, counter.items())
        sorted_word_to_cnt = sorted(counter,key=itemgetter(1),reverse=True)
        sorted_words = [x[0] for x in sorted_word_to_cnt]

    sorted_words = ["<unk>","<start>","<end>"] + sorted_words

    print("vocab_len: " + str(len(sorted_words)))

    return sorted_words

In [15]:
input_data = "../00-data/tf_data.txt"
min_word_freq = 0
vocab = get_vocab(input_data,min_word_freq)
vocab[70:80]

vocab_len: 11928


['in_channels',
 'zeros',
 'class',
 'plt',
 'session',
 'variable_scope',
 'join',
 'size',
 'Session',
 'zip']

In [8]:
word_dict = {word:index for index,word in enumerate(vocab)}

In [10]:
word_dict["Session"]

78

## 【随机】前k句预测下一句

In [5]:
def clean_and_split(line):
    line = ' '.join(re.split(' |\t|\v|\n',line))        
    line = re.split('([: ,.(){}\[\]=])',line)        
    line = list(filter(lambda x: x!=' 'and x!='',line))
    return line

In [6]:
def get_newdata_by_random(raw_data,vocab,enc_vec_data,enc_text_data,dec_vec_data,dec_text_data,rand_max=10,duplicate=2):
    import codecs
    import sys
    import re

    word_to_id = {k:v for(k,v) in zip(vocab,range(len(vocab)))}

    def get_id(word):
        return word_to_id[word] if word in word_to_id else word_to_id["<unk>"]

    fout_vec_enc = codecs.open(enc_vec_data,"w","utf-8")
    fout_vec_dec = codecs.open(dec_vec_data,"w","utf-8")
    fout_text_enc = codecs.open(enc_text_data,"w","utf-8")
    fout_text_dec = codecs.open(dec_text_data,"w","utf-8")
    
    data_length = 0
    with open(raw_data,"r") as fin:
        lines = fin.readlines()
        for i in range(rand_max,len(lines)):
            rand_nums = set(random.randint(1,rand_max) for _ in range(duplicate))
            for rand_num in rand_nums:
                #构造enc_data
                words = []
                for j in range(i - rand_num,i):
                    line = clean_and_split(lines[j])
                    words += ["<start>"] + line + ["<end>"]
                out_line = ' '.join([str(get_id(w)) for w in words]) + '\n'
                fout_text_enc.write(' '.join(words) + '\n')
                fout_vec_enc.write(out_line)

                #构造dec_data
                words = []
                line = clean_and_split(lines[i])
                words = line + ["<end>"]
                out_line = ' '.join([str(get_id(w)) for w in words]) + '\n'
                fout_text_dec.write(' '.join(words) + '\n')
                fout_vec_dec.write(out_line)
            
                data_length += 1
    fout_vec_enc.close()
    fout_vec_dec.close()
    fout_text_enc.close()
    fout_text_dec.close()
    
    return data_length

In [7]:
enc_vec_data = "../00-data/train.enc"
dec_vec_data = "../00-data/train.dec"
enc_text_data = "../00-data/train_text.enc"
dec_text_data = "../00-data/train_text.dec"

In [8]:
data_length = get_newdata_by_random(input_data,vocab,enc_vec_data,enc_text_data,dec_vec_data,dec_text_data)
print("data_len: " + str(data_length))

data_len: 74052


## 原始数据向量化

In [9]:
def MakeDataset(file_path):
    dataset = tf.data.TextLineDataset(file_path)
    dataset = dataset.map(lambda string:tf.string_split([string]).values)
    dataset = dataset.map(
            lambda string: tf.string_to_number(string,tf.int32))
    dataset = dataset.map(lambda x:(x,tf.size(x)))
    return dataset

In [10]:
def MakeSrcTrgDataset(src_path,trg_path,batch_size,shuffle_size=3000,start_id=1):
    src_data = MakeDataset(src_path)
    trg_data = MakeDataset(trg_path)
    
    dataset = tf.data.Dataset.zip((src_data,trg_data))
    
    #删除内容为空or长度过长的句子
    #不需要执行
    def FilterLength(src_tuple,trg_tuple):
        ((src_input,src_len),(trg_label,trg_len)) = (src_tuple,trg_tuple)
        src_len_ok = tf.logical_and(
            tf.greater(src_len,1),tf.less_equal(src_len,MAX_LEN))
        trg_len_ok = tf.logical_and(
            tf.greater(trg_len,1),tf.less_equal(trg_len,MAX_LEN))
        return tf.logical_and(src_len_ok,trg_len_ok)
    #dataset = dataset.filter(FilterLength)
    
    #生成<start> X Y Z 作为解码器的输入
    def MakeTrgInput(src_tuple,trg_tuple):
        ((src_input,src_len),(trg_label,trg_len)) = (src_tuple,trg_tuple)
        trg_input = tf.concat([[start_id],trg_label[:-1]],axis=0)
        return ((src_input,src_len),(trg_input,trg_label,trg_len))
    dataset = dataset.map(MakeTrgInput)
    
    #随机打乱训练数据
    dataset = dataset.shuffle(shuffle_size)
    
    #规定填充后输出的数据维度
    padded_shapes = (
        (tf.TensorShape([None]),
         tf.TensorShape([])),
        (tf.TensorShape([None]),
         tf.TensorShape([None]),
         tf.TensorShape([])))
    #调用padded_batch方法进行batching操作
    batched_dataset = dataset.padded_batch(batch_size,padded_shapes)
    return  batched_dataset

## seq2seq + attention模型
encoder:2层单向lstm
decoder:1层单向lstm

In [11]:
CHECKPOINT_PATH = "../02-checkpoints/"
HIDDEN_SIZE = 256
ENCODER_LAYERS = 2
DECODER_LAYERS = 1
SRC_VOCAB_SIZE = len(vocab)
TRG_VOCAB_SIZE = len(vocab)
BATCH_SIZE = 64
NUM_EPOCH = 10
KEEP_PROB = 0.9
MAX_GRAD_NORM = 5
LEARNING_RATE_BASE = 1.0
LEARNING_RATE_DECAY = 0.7
SHARE_EMB_AND_SOFTMAX = True

In [12]:
# 定义NMTModel类来描述模型。
class NMTModel(object):
    # 在模型的初始化函数中定义模型要用到的变量。
    def __init__(self):
        # 定义编码器和解码器所使用的LSTM结构。
        self.enc_cell_fw = tf.nn.rnn_cell.BasicLSTMCell(HIDDEN_SIZE)
        self.enc_cell_bw = tf.nn.rnn_cell.BasicLSTMCell(HIDDEN_SIZE)
        
        self.enc_cell = tf.nn.rnn_cell.MultiRNNCell(
          [tf.nn.rnn_cell.BasicLSTMCell(HIDDEN_SIZE) 
           for _ in range(ENCODER_LAYERS)])
        
        self.dec_cell = tf.nn.rnn_cell.MultiRNNCell(
          [tf.nn.rnn_cell.BasicLSTMCell(HIDDEN_SIZE) 
           for _ in range(DECODER_LAYERS)])

        # 为源语言和目标语言分别定义词向量。   
        self.src_embedding = tf.get_variable(
            "src_emb", [SRC_VOCAB_SIZE, HIDDEN_SIZE])
        self.trg_embedding = tf.get_variable(
            "trg_emb", [TRG_VOCAB_SIZE, HIDDEN_SIZE])

        # 定义softmax层的变量
        if SHARE_EMB_AND_SOFTMAX:
            self.softmax_weight = tf.transpose(self.trg_embedding)
        else:
            self.softmax_weight = tf.get_variable(
               "weight", [HIDDEN_SIZE, TRG_VOCAB_SIZE])
        self.softmax_bias = tf.get_variable(
            "softmax_bias", [TRG_VOCAB_SIZE])

    # 在forward函数中定义模型的前向计算图。
    # src_input, src_size, trg_input, trg_label, trg_size分别是上面
    # MakeSrcTrgDataset函数产生的五种张量。
    def forward(self, src_input, src_size, trg_input, trg_label, trg_size,data_length):
        global_step = tf.Variable(0, trainable=False)
        batch_size = tf.shape(src_input)[0]
    
        # 将输入和输出单词编号转为词向量。
        src_emb = tf.nn.embedding_lookup(self.src_embedding, src_input)
        trg_emb = tf.nn.embedding_lookup(self.trg_embedding, trg_input)
        
        # 在词向量上进行dropout。
        src_emb = tf.nn.dropout(src_emb, KEEP_PROB)
        trg_emb = tf.nn.dropout(trg_emb, KEEP_PROB)

        # 使用dynamic_rnn构造编码器。
        # 编码器读取源句子每个位置的词向量，输出最后一步的隐藏状态enc_state。
        # 因为编码器是一个双层LSTM，因此enc_state是一个包含两个LSTMStateTuple类
        # 张量的tuple，每个LSTMStateTuple对应编码器中的一层。
        # 张量的维度是 [batch_size, HIDDEN_SIZE]。
        # enc_outputs是顶层LSTM在每一步的输出，它的维度是[batch_size, 
        # max_time, HIDDEN_SIZE]。Seq2Seq模型中不需要用到enc_outputs，而
        # 后面介绍的attention模型会用到它。
        # 下面的代码取代了Seq2Seq样例代码中forward函数里的相应部分。
        with tf.variable_scope("encoder"):
            # 构造编码器时，使用dynamic_rnn构造单向循环网络。
            # 单向循环网络的顶层输出enc_outputs是一个包含两个张量的tuple，每个张量的
            # 维度都是[batch_size, max_time, HIDDEN_SIZE]，代表两个LSTM在每一步的输出。
            enc_outputs,enc_state = tf.nn.dynamic_rnn(
                self.enc_cell,src_emb,src_size,dtype=tf.float32)    

        with tf.variable_scope("decoder"):
            # 选择注意力权重的计算模型。BahdanauAttention是使用一个隐藏层的前馈神经网络。
            # memory_sequence_length是一个维度为[batch_size]的张量，代表batch
            # 中每个句子的长度，Attention需要根据这个信息把填充位置的注意力权重设置为0。
            attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
                HIDDEN_SIZE, enc_outputs,
                memory_sequence_length=src_size)

            # 将解码器的循环神经网络self.dec_cell和注意力一起封装成更高层的循环神经网络。
            attention_cell = tf.contrib.seq2seq.AttentionWrapper(
                self.dec_cell, attention_mechanism,
                attention_layer_size=HIDDEN_SIZE)

            # 使用attention_cell和dynamic_rnn构造编码器。
            # 这里没有指定init_state，也就是没有使用编码器的输出来初始化输入，而完全依赖
            # 注意力作为信息来源。
            dec_outputs, _ = tf.nn.dynamic_rnn(
                attention_cell, trg_emb, trg_size, dtype=tf.float32)

        # 计算解码器每一步的log perplexity。这一步与语言模型代码相同。
        output = tf.reshape(dec_outputs, [-1, HIDDEN_SIZE])
        logits = tf.matmul(output, self.softmax_weight) + self.softmax_bias
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=tf.reshape(trg_label, [-1]), logits=logits)

        # 在计算平均损失时，需要将填充位置的权重设置为0，以避免无效位置的预测干扰
        # 模型的训练。
        label_weights = tf.sequence_mask(
            trg_size, maxlen=tf.shape(trg_label)[1], dtype=tf.float32)
        label_weights = tf.reshape(label_weights, [-1])
        cost = tf.reduce_sum(loss * label_weights)
        cost_op = cost / tf.reduce_sum(label_weights)
        
        # 定义反向传播操作。反向操作的实现与语言模型代码相同。
        trainable_variables = tf.trainable_variables()

        # 控制梯度大小，定义优化方法和训练步骤。
        grads = tf.gradients(cost / tf.to_float(batch_size),
                             trainable_variables)
        grads, _ = tf.clip_by_global_norm(grads, MAX_GRAD_NORM)
        
        learning_rate = tf.train.exponential_decay(
            LEARNING_RATE_BASE,
            global_step,
            data_length / batch_size, 
            LEARNING_RATE_DECAY,
            staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        train_op = optimizer.apply_gradients(
            zip(grads, trainable_variables))
        return cost_op, train_op,learning_rate,global_step

In [13]:
def run_epoch(session,cost_op,train_op,learning_rate_op,global_step_op,step,saver,epoch):
    while True:
        try:
            cost,_,learning_rate,global_step = session.run([cost_op,train_op,learning_rate_op,global_step_op])
            if global_step % 50 == 0:
                print("After %d global_steps,per token cost is %.3f,learning_rate is %.5f" %(global_step,cost,learning_rate))
            session.run(tf.assign(global_step_op,step))
            step += 1
        except tf.errors.OutOfRangeError:
            if epoch % 2 == 0:
                saver.save(session,CHECKPOINT_PATH,global_step=global_step)
            break
    return step

In [ ]:
def main():
    tf.reset_default_graph()
    initializer = tf.random_uniform_initializer(-0.05,0.05)
    with tf.variable_scope("nmt_model",reuse=None,initializer=initializer):
        train_model = NMTModel()
    
    data = MakeSrcTrgDataset(enc_vec_data,dec_vec_data,BATCH_SIZE)
    iterator = data.make_initializable_iterator()
    (src,src_size),(trg_input,trg_label,trg_size) = iterator.get_next()
    
    cost_op,train_op,learning_rate_op,global_step_op = train_model.forward(src,src_size,trg_input,trg_label,trg_size,data_length)
    
    saver = tf.train.Saver()
    step = 1
    
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        ckpt = tf.train.get_checkpoint_state(CHECKPOINT_PATH) #获取checkpoints对象  
        if ckpt and ckpt.model_checkpoint_path:##判断ckpt是否为空，若不为空，才进行模型的加载，否则从头开始训练  
            saver.restore(sess,ckpt.model_checkpoint_path)#恢复保存的神经网络结构，实现断点续训 
        for i in range(NUM_EPOCH):
            print("In EPOCH: %d" %(i + 1))
            sess.run(iterator.initializer)
            step = run_epoch(sess,cost_op,train_op,learning_rate_op,global_step_op,step,saver,i + 1)
if __name__ == "__main__":
    main()

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring paramet